 1. Mach–Zehnder Interferometer (MZI) Simulation

    Quantum Circuit Approach (Qiskit):

        Encode MZI using qubit gates: Hadamards for beam splitters, Rz(ϕ)Rz​(ϕ) for phase shift.

        Benchmark interference fringes by varying phase φ.

    Boson Sampling (Unitary) Simulation:

        Model MZI as a 2×2 unitary UMZI=BS⋅P(ϕ)⋅BSUMZI​=BS⋅P(ϕ)⋅BS.

        Apply directly to mode amplitudes (classical simulation of photonic interference).

    Extension:

        Build multi-layer interferometer arrays (cascaded MZIs), analogous to Galton boards.

        Compare qubit-circuit vs bosonic-mode simulation fidelity.

 2. Classical-Quantum Equivalence Benchmarking

    Goal: Show that Galton boards, Mach–Zehnder interferometers, and Ramsey interferometry are all formally equivalent interference processes.

    Method:

        Map Galton scattering (CSWAP layers) to beam splitters.

        Demonstrate distribution equivalence: Binomial (Galton) ↔ Interference fringes (MZI) ↔ Ramsey oscillations.

 3. Coherence-Stabilized Ramsey Interferometry

    Implement coherence stabilization protocol (from the paper):

        Apply Trotterized evolution under H(t)=Δσz/2+hy(t)σy/2H(t)=Δσz​/2+hy​(t)σy​/2.

        Dynamically update hy(t)hy​(t) from Bloch vector components.

    Benchmark:

        Compare stabilized vs standard Ramsey sequences for sensitivity and SNR.

        Test robustness with noise models (T1/T2 decoherence).

 4. Hybrid Stabilization in Galton Boards

    Key Idea: Treat the Galton board’s control qubit as analogous to the Ramsey qubit.

    Method:

        Between scattering layers, insert stabilization pulses (Trotterized RzRz​ and RyRy​ gates).

        Dynamically compute hy(t)hy​(t) from the control qubit’s Bloch state (feedback-based correction).

    Outcome:

        Maintain ideal distributions over deeper boards under noise.

        Directly connect sensing control techniques to interferometric transport models.

^ what chatgpt made of my plan, not quite there but close enough for now.